# Group Mahalanobis Project 1


## Topic & Goals

For Project 1, our group will focus on understanding population trends in various Southeast Asian countries as it relates to urbanization, globalization and economic development.

Our group is interested in analyzing multiple variables over time (20th century to today) including:<br>
Demographics<br>
Economic output<br>
Agricultural factors of production<br>
GDP and population Analysis<br>

## Project Deliverables

### [#A] Data Imports

In [ ]:
"""
If necessary, uncomment and install:
"""
#!pip install wbdata
#!pip install cufflinks
#!pip install iso3166

import iso3166 #iso3166.countries.get('country details')
import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
cf.go_offline()

We are using data from **Source 40: "Population Estimates and Projections"**. We can also use **wbdata.get_topic()** and **wbdata.get_indicator()** to explore other datasets.

WBDATA API ref: https://wbdata.readthedocs.io/en/stable/wbdata_library.html

In [ ]:
#wbdata.get_source() to get all sources
#wbdata.get_topic()
#wbdata.get_indicator()

source_id = 40 #Population Estimates and Projections
indicators = wbdata.get_indicator(source=source_id)

### [#A] Population DataFrames

A function that returns a pandas DataFrame
indexed by Region or Country and Year, with
columns giving counts of people in different
age-sex groups:

**pop_df (year, group, age_lower, age_upper, location)**

In [ ]:
"""
population ranges available from dataset:
00-04, 05-09, 10-14, 15-19, 20-24, 25-29, 30-34, ..., 70-74, 75-79, above 80
"""

def pop_df(year='2021', group='all', age_lower=0, age_upper=100, location='world'):
    df = pop_df_helper(year, age_lower, age_upper, location)
        
    if group == 'males':
        return df.drop(columns = ['Female'])
    elif group == 'females':
        return df.drop(columns = ['Male'])
    else:
        total_pop = df["Female"] + df["Male"]
        df["Total"] = total_pop
    return df

# Returns a list of input strings for population age ranges
def pop_df_helper(year, age_lower, age_upper, location):
    if location != 'world':
        country_alpha3 = iso3166.countries.get(location).alpha3
        country_name = iso3166.countries.get(location).apolitical_name
    else:
        country_alpha3 = 'WLD'
        country_name = 'World'
    inputs = list_of_age_inputs(age_lower, age_upper)

    #create indicator-dictionaries required for WBData API
    indicator_dict_m = {}
    indicator_dict_f = {}
    for i in inputs:
        key_m = "SP.POP.{}.{}".format(i, 'MA')
        key_f = "SP.POP.{}.{}".format(i, 'FE')
        value = "{}-{}".format(i[:2], i[2:])
        indicator_dict_m[key_m] = value
        indicator_dict_f[key_f] = value

    #source_id = 40, refer to wbdata.get_source()
    wbdf_m = wbdata.get_dataframe(indicator_dict_m, country=country_alpha3, source=source_id)
    wbdf_f = wbdata.get_dataframe(indicator_dict_f, country=country_alpha3, source=source_id)
    datas_m = wbdf_m.query("date=='{}'".format(year)).sum(axis=0).tolist()
    datas_f = wbdf_f.query("date=='{}'".format(year)).sum(axis=0).tolist()
    df = pd.DataFrame({
        'Country': country_name,
        'Year': year,
        'Age': list(indicator_dict_m.values()),
        'Female': datas_f,
        'Male': datas_m
    })
    return df

# Returns a list of input strings for population age ranges
def list_of_age_inputs(age_lower, age_upper):
    def round_down(n):
        return max(0, n - n % 5)

    def round_up(n):
        return n - n % 5 + 5

    results = []
    r_lower_bound, r_upper_bound = round_down(age_lower), round_up(age_upper)   
    while r_lower_bound < min(79, r_upper_bound):
        results.append("{:02d}{:02d}".format(r_lower_bound, r_lower_bound + 4))
        r_lower_bound += 5
    if age_upper >= 80:
        results.append('80UP')
    return results

The **pop_df** function takes in the year, target group, age lower/upper bounds and the country as arguments. The function returns a dataframe.

Variables:

In [ ]:
year = 2015 #1960 to 2050 (projections)
group = 'all' #males/females/all
age_lower = 13 #rounds to nearest 5 (actual range: 10 to 65)
age_upper = 61
location = 'myanmar' #accepts country names/abbreviations, 'world'

Function call and output:

In [ ]:
df = pop_df(year, group, age_lower, age_upper, location)

A dataframe is returned with the above format. The returned dataframe can be used to construct visualizations.

### [#A] Population Statistics

A python function named population
that will deliver information to
answer the queries of the following
form:

In [year] how many
[people/males/females] aged [low] to
[high] were living in [the
world/region/country]?

**pop_stats (year, group, age_lower, age_upper, location)**

In [ ]:
def pop_stats(year='2021', group='all', age_lower=0, age_upper=100, location='world'):
    df = pop_df(year, group, age_lower, age_upper, location)
    inputs = list_of_age_inputs(age_lower, age_upper);
    total = df.sum()[3]
    t = total.astype(int)
    age_l = inputs[0][0:2]
    
    if group == 'people':
        g = 'people'
    else:
        g = group
        
    if age_upper >= 80:
        age_h = '80 or over'
    else:
        age_h = inputs[-1][2:4]
    
    if location == 'world':
        loc = 'the world'
    else:
        loc = iso3166.countries.get(location).apolitical_name
    
    if year > 2021:
        print("By estimation, in {}, there will be {} {} aged {} to aged {} living in {}.".format(year, t, g, age_l, age_h, loc))
    else:
        print("In {}, there are {} {} aged {} to aged {} living in {}.".format(year, t, g, age_l, age_h, loc))
        

Our **pop_stats** function takes in the year, target group, age lower/upper bounds and the country as arguments. The function returns a summary containing the information we need.

Variables:

In [ ]:
year = 2021
group = 'males'
age_lower = 10
age_upper = 45
location = 'world'

Function call and output:

In [ ]:
#pop_stats(year, group, age_lower, age_upper, location)

### [#B] Population Pyramids

A python function that takes as
input a pandas DataFrame with
columns providing counts of people by
age-sex groups, and constructs a
“population pyramid” graph for
visualizing the data.

**draw_pop_pyramid (dataframe)**

In [ ]:
def draw_pop_pyramid(df):
    #negate values under 'Male'
    df['Male'] = df['Male'].apply(lambda x: -x)
    #remove unnecessary columns
    df.drop(columns = ['Country', 'Year', 'Total'])

    AgeClass = df['Age'][::-1]
    bar_plot = sns.barplot(x='Female', y='Age', data=df, order=AgeClass)
    bar_plot = sns.barplot(x='Male', y='Age', data=df, order=AgeClass)
    bar_plot.set(xlabel="Population (millions)", ylabel="Age-Group", title = "Population Pyramid")

**draw_pop_pyramid** function call and output:

In [ ]:
year = 2018
group = 'all'
age_lower = 0
age_upper = 80
location = 'thailand'

df = pop_df(year, group, age_lower, age_upper, location)
draw_pop_pyramid(df) 

#### Functions we have so far:
**pop_df(year, group, age_lr, age_up, location)**: returns a dataframe for visualizations<br>
**pop_stats(year, group, age_lr, age_up, location)**: returns a print statement containing population information<br>
**draw_pop_pyramid(dataframe)**: accepts a dataframe created using pop_df(), and creates a population histogram

### [#C] Animated Population Pyramids (Thailand as an example)

A python function that takes as
input a pandas DataFrame with
columns providing counts of people by
age-sex groups, with rows
corresponding to different years, and
constructs an animated “population
pyramid” graph for visualizing how the
population changes over time.

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

In [ ]:
# which take the form "SP.POP.LLHH.MA" for males
# and "SP.POP.LLHH.FE" for females, where LL is the *low* end of
# age range, like "05" for 5-yo, and HH is the *high* end.
# We construct a list of age-ranges.
# Start with an empty list of age-rages
age_ranges = []

# Ranges top out at 80, and go in five year increments
for i in range(0,80,5):
    age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

age_ranges.append("80UP")

print(age_ranges)

In [ ]:
male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
variables = male_variables
variables.update(female_variables)

In [ ]:
df = wbdata.get_dataframe(variables,country="THA", cache = False)

In [ ]:
df.head()

In [ ]:
import plotly.graph_objs as go
import ipywidgets
from ipywidgets import interactive, HBox, VBox

def helper(year):
    py.init_notebook_mode(connected=True)
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))
    bins = [go.Bar(x = df.loc[str(year),:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='green'),
               hoverinfo='skip'
               ),
        go.Bar(x = -df.loc[str(year),:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='pink'),
               hoverinfo='skip',
               )
        ]
    py.iplot(dict(data=bins, layout=layout))
ipywidgets.interact(helper, year=(1980, 2018, 1))

### [#C] Population Maps (Indonesia as an example)

A Jupyter notebook which queries a
population database and provides
visualizations in the form of maps.
An example of a library that may be
useful is geopandas.

In [ ]:
#!pip install geopandas
import geopandas

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
fp = "IDN_adm/IDN_adm1.shp"
map_df = gpd.read_file(fp)
plt.rcParams['figure.figsize'] = [30, 50] 
map_df.plot();

In [ ]:
province = pd.read_csv("data_province.csv", sep=";")
merged = map_df.merge(province, how='left', left_on="NAME_1", right_on="province")
merged = merged[['province', 'geometry', 'population_2015', 'area_km2', 'population_density_per_km2',
                'cities_regencies', 'cities', 'regencies']]
variable = 'cities_regencies'
vmin, vmax = 0, 60
fig, ax = plt.subplots(1, figsize=(30, 10))
ax.axis('off')
ax.set_title('Population Density Map', fontdict={'fontsize': '25', 'fontweight' : '3'})
ax.annotate('Source: Wikipedia - https://en.wikipedia.org/wiki/Provinces_of_Indonesia',xy=(0.6, .05),
            xycoords='figure fraction', fontsize=12, 
            color='#555555')
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([]) 
fig.colorbar(sm)
merged.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')
merged['coords'] = merged['geometry'].apply(lambda x: x.representative_point().coords[:])
merged['coords'] = [coords[0] for coords in merged['coords']]
for idx, row in merged.iterrows():
    plt.annotate(text=row['province'], xy=row['coords'],horizontalalignment='center')

In [ ]:
#A function that draws a geopanda graph, accepting a query_dictionary, the year and the title as arguments

def draw_geopanda_sea(indi_dict, year, title):
    countries = ['Myanmar', 'Thailand', 'Vietnam', 'Cambodia', 'Malaysia', 'Indonesia', 
                 "Philippines", "Brunei"]
    countries_iso3166 = ['MMR', 'THA', "VNM", 'KHM', 'MYS', 'IDN', 'PHL', 'BRN']
    
    df = wbdata.get_dataframe(indicators=indi_dict, country=countries_iso3166)
    df.reset_index(inplace=True)
    df['date'] = df['date'].astype(int)
    df = df[df['date'] == year]
    df = df.sort_values(by=['country'])

    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    world = world[world['name'].isin(countries)]
    world = world.sort_values(by=['name'])
    world['data_column'] = df['data_column'].values

    graph = geopandas.GeoDataFrame.plot(world, 'data_column', cmap='Spectral', figsize=(14, 14), legend=True, 
                                        legend_kwds={'shrink': 0.65})
    
    graph.set_title(title, size=25, pad=20);

    world.apply(lambda x: graph.annotate(text=x['name'], xy=x.geometry.centroid.coords[0], 
                                         ha='center'),axis=1);

In [ ]:
#indicator: Population Growth Rate (%)
indi_dict = {'SP.POP.GROW': 'data_column'}
draw_geopanda_sea(indi_dict, 2019, 'Population Growth Rate 2019 (%)')

In [ ]:
#indicator: GDP per capita growth (annual %)
indi_dict = {'NY.GDP.PCAP.KD.ZG': 'data_column'}
draw_geopanda_sea(indi_dict, 2019, 'GDP per capita growth 2019 (%)')

**Correlation Calculation**

In [ ]:
def get_correlation_sea(two_indicators_dict):
    countries_iso3166 = ['MMR', 'THA', "VNM", 'KHM', 'MYS', 'IDN', 'PHL', 'BRN']
    df = wbdata.get_dataframe(two_indicators_dict, country=countries_iso3166)
    df.reset_index(inplace=True)
    df['date'] = df['date'].astype(int)
    df = df[df['date'] == year]
    df = df.sort_values(by=['country'])
    df = df[['gdpppc', 'pop_growth']]
    return df.corr()

In [ ]:
two_indicators_dict = {'NY.GDP.PCAP.KD.ZG': 'gdpppc', 'SP.POP.GROW': 'pop_growth'}    
get_correlation_sea(two_indicators_dict)

### [#C] Other Visualization Tools

In [27]:
# Graphing Population of Southeast Asian Countries Over Time (1961-2020) using World Band data

se_countries = {'BRN': 'Brunei Darussalam',
                'KHM': 'Cambodia',
                'IDN': 'Indonesia',
                'LAO': 'Laos',
                'MYS': 'Malaysia',
                'MMR': 'Myanmar',
                'PHL': 'Philippines',
                'THA': 'Thailand',
                'VNM': 'Viet Nam',
                'SGP': 'Singapore',
               }

variable_labels = {"SP.POP.TOTL":"World Population"}
se_population = wbdata.get_dataframe(variable_labels, country = se_countries).squeeze()

se_population = se_population.unstack('country')


se_population.iplot(title="Population of Southeast Asian Countries Over Time (1961-2020)",xTitle='Year',yTitle='Population')

In [28]:
# Graphing GDP per Capita of Southeast Asian Countries Over Time
gdp_indicator = {"NY.GDP.PCAP.CD":"GDP per capita"} 

gdp = wbdata.get_dataframe(gdp_indicator, country = se_countries).squeeze().unstack('country')
gdp.iplot(title="GDP per Capita of Southeast Asian Countries Over Time",xTitle='Year',yTitle='GDP per Capita (current US$)')

In [ ]:
group = 'all' #males/females/all
age_lower = 0 #rounds to nearest 5 (actual range: 10 to 65)
age_upper = 79
location = 'Singapore' #accepts country names/abbreviations, 'world'

years = [1980, 1990, 2000, 2010, 2020]
frames = []
for y in years:
    frames.append(pop_df(y, group, age_lower, age_upper, location))
    
result = pd.concat(frames, axis=0, ignore_index = True)
result = result.drop(columns = ['Male', 'Female', 'Country'])
result = result.pivot(index='Year', columns='Age')

for c in result:
    result[c] = np.log(result[c])

my_colors = [(x/25, x/25, x/20, 0.75) for x in range(17)] 
graph = result.plot(kind='bar', figsize=(20,10), ylim=(10, 13.2), legend='reverse', rot=0, alpha=0.8, color=my_colors)
handles, labels = graph.get_legend_handles_labels()
graph.set_ylabel('Total Population (Log)');
graph.set_title("Population Change in {} over decades".format(location))
graph.legend(handles[::-1], labels[::-1], title='Age Groups', loc='center left', bbox_to_anchor=(1.0, 0.5));

In [ ]:
group = 'all' #males/females/all
age_lower = 0 #rounds to nearest 5 (actual range: 10 to 65)
age_upper = 85
location = 'Cambodia' #accepts country names/abbreviations, 'world'

years = [1980, 1990, 2000, 2010, 2020]
frames = []
for y in years:
    frames.append(pop_df(y, group, age_lower, age_upper, location))
    
result = pd.concat(frames, axis=0, ignore_index = True)
result = result.drop(columns = ['Male', 'Female', 'Country'])
result = result.pivot(index='Year', columns='Age')

for c in result:
    result[c] = np.log(result[c])

my_colors = [(x/30, x/30, 0.75) for x in range(17)] 
graph = result.plot(kind='bar', figsize=(20,10), ylim=(10, 14.5), legend='reverse', rot=0, alpha=0.8, color=my_colors)
handles, labels = graph.get_legend_handles_labels()
graph.set_ylabel('Total Population (Log)');
graph.set_title("Population Change in {} over decades".format(location))
graph.legend(handles[::-1], labels[::-1], title='Age Groups', loc='center left', bbox_to_anchor=(1.0, 0.5));

### [#C] Agricultural Supply

Relate changes in agricultural food supply
to changes in population.

What would Malthus think?

In [ ]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot
from scipy.optimize import newton

init_notebook_mode(connected=True)

# Play with these constants
## for exponential
P0 = 1
r = 0.02

## for line
F0 = 10
m = 100

food_production = lambda t,m=m: F0+m*t
population = lambda t,r=r: P0*np.exp(r*t)
diff = lambda t,r=r,m=m: food_production(t,m) - population(t,r)
# Point where curves cross
def choose_xaxis(r,m,x0=10):
    """
    Find a range of x so that curves intersect
    """
    tstar = -1
    try:
        while tstar <= 0:
            tstar = newton(lambda t: diff(t,r,m),x0,maxiter=1000)
            x0 = x0*10
        return np.linspace(0,tstar*1.1,100)
    except RuntimeError: # Failed to find root
        return np.linspace(0,1000,100)

In [ ]:
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox

T = choose_xaxis(r,m)

data = [dict(name='Food production',
             #visible = False,
             line=dict(color='blue', width=6),
             x = T,
             y = [food_production(t) for t in T]),
        dict(name='Population',
             #visible = False,
             line=dict(color='red', width=6),
             x = T,
             y = [population(t) for t in T])]

layout = {}
#fig = dict(data=data, layout=layout)
#plot(fig)
f = go.FigureWidget(data=data,layout=layout)
def update_pop(r,m):
    T = choose_xaxis(r,m)
    f.data[1].y = [population(t,r) for t in T]
    f.data[0].x = T
    f.data[1].x = T
r_slider = interactive(update_pop,r=(-0.05,0.08,.01),m=(0,200,10))
vb = VBox((f, r_slider))
vb.layout.align_items = 'center'
vb

In [ ]:
import numpy as np
import wbdata
import cufflinks as cf
cf.go_offline()

def fix_date_index(df):
    idx_vars = df.index.names
    new = df.reset_index()
    new.date = new.date.astype(int)
    return new.set_index(idx_vars)

In [ ]:
vars = {"AG.PRD.CREL.MT":"Cereal production"}
use=["CHN","USA","IND","FRA","RUS","WLD"]
cereals = fix_date_index(wbdata.get_dataframe(vars,country=use)).squeeze().unstack('country')
cereals.iplot(xTitle="Year",yTitle="Metric Tons",
              title="Cereal Production by Top 5 (in 2000)")

In [ ]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index"}
food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))
food.iplot(xTitle="Year",yTitle="Index")

In [ ]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index",
        "SP.POP.TOTL":"Population"}
food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))
food.sort_index(inplace=True)
food['Population'] = food['Population']/food.loc[2006,'Population']*100
food.iplot(xTitle="Year",yTitle="Index")

In [ ]:
np.log(food).diff().iplot(xTitle="Year",yTitle="Growth rates")

## Further/Others (Linear Regression, Cambodia as an example)
### A simple regression (X: population, y: cereal production)

In [ ]:
#Vietnam Cereal Production by year
cmfood = fix_date_index(wbdata.get_dataframe(vars,country='KHM'))
cmfood2 = vtfood.dropna()['Cereal production'].tolist()

In [ ]:
#Vietnam population by year
#df = wbdata.get_dataframe(variables, country="KHM")
popcm = df.sum(axis=1) 
popcm2 = popcm.dropna().tolist()
popcm3 = popcm2[:57]

In [ ]:
import statsmodels.api as sm
X = popcm3
y = cmfood2
model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
#Predict Cereal production at 2030
prediction_2030 = model.predict(2030)
prediction_2030